In [27]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset

In [24]:
rating_files = ['Japan_Patient7_PainRatingTask_InsulaB49.csv',
                'Japan_Patient7_PainRatingTask_InsulaB50.csv',
                'Japan_Patient7_PainRatingTask_InsulaB51.csv',
                'Japan_Patient7_PainRatingTask_InsulaB52.csv',
               ]

In [31]:
# Convert the csv files into numpy array
#col_names = [f'BeforeXpression_{i}' for i in range(1000)] + [f'AfterXpression_{i}' for i in range(1000)] + ['StimType', 'StimIntensity', 'SubjRating' ]
ntrials = 252
ncols = 2003
nelectrodes = 4

all_data = np.zeros((ntrials, ncols, nelectrodes))
for i, f in enumerate(rating_files):
    all_data[:,:,i] = pd.read_csv(file_rating, header=None).values

np.save('patient7.npy', all_data)

#df.index = [f'Trial_{i}' for i in range(252)]

In [36]:
all_data[:,range(10,20),:].shape

(252, 10, 4)

In [39]:
class ecogDataset(Dataset):
    def __init__(self, data_file, features='afterX', label='type', split='train'):
        '''
        data_file = '.npy file [ntrials X ncols[all features + labels] X nelectrodes ]'
        features = 'beforeX' | 'afterX' | 'all'
        label = 'type' | 'intensity' | 'rating'
        split = 'train' | 'val' | 'test'
        '''
        
        data = np.load(ecog_response_file)
        ntrials, _, _ = data.shape
        
        ntrain = int(0.8 * ntrials)
        nval = int(0.1 * ntrials)
        
        if features == 'beforeX':
            select_features = range(0, 1000)
        if features == 'afterX':
            select_features = range(1000, 2000)
        if features == 'all':
            select_features = range(0, 2000)
                
        if label == 'type':
            select_label = 2001
        if label == 'intensity':
            select_label = 2002
        if label == 'rating':
            select_label = 2003
        
        if split == 'train':
            select_trials = range(0, ntrain)
        if split == 'val':
            select_trials = range(ntrain, ntrain+nval)
        if split == 'test':
            select_trials = range(ntrain+nval, ntrials)
        
        
        self_features = data[select_trials, select_features, :]
        self_label = data[select_label]    
    
        
        
    def __len__(self):
        return self_features.shape[0] # num trials
    
    def __getitem__(self, index):
        return self.features[index], self.label[index]
        
        

In [15]:
class Rating1DConvNet(nn.Module):
    def __init__(self, nROIS=2):
        super(Rating1DConvNet, self).__init__()
        
        self.conv1 = nn.Conv1d(nROIS, 16, 7)
        self.conv2 = nn.Conv1d(16,32, 5)
        self.conv3 = nn.Conv1d(32, 64, 5)
        self.avg = nn.AdaptiveAvgPool1d((1))
        
        self.linear1 = nn.Linear(64, 100)
        self.linear2 = nn.Linear(100, 2)
        
    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.avg(x).view(-1, 64)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        
        return x
    

In [22]:
def validate_model(net, val_data, analysis_type, nTime_min=150, criterion=nn.CrossEntropyLoss()):
    
    val_data_loader = DataLoader(val_data, batch_size=256, shuffle=True)
    
    net.eval()
    loss = 0.0
    for i, (tc, corr, dx) in enumerate(val_data_loader):

            if analysis_type == 'tc':
                vals = Variable(tc).type(torch.FloatTensor)
            if analysis_type == 'corr':
                vals = Variable(corr).type(torch.FloatTensor)

            dx = Variable(dx).type(torch.LongTensor)
            
            # forward pass
            output = net(vals)

            # calculate loss
            loss += criterion(output, torch.max(dx,1)[1])

    return loss/len(val_data_loader)

In [40]:
def train_network(atlas_name, bptf, confounds, analysis_type ='tc', nTime_min=150, nepochs=100, verbose=True):
    
    
    train_data = AbideData(atlas_name=atlas_name, 
                           bptf=bptf, 
                           confounds=confounds,
                           nTime_min=nTime_min, split='train')
    train_data_loader = DataLoader(train_data, batch_size=128, shuffle=True)
    
    val_data = AbideData(atlas_name=atlas_name, nTime_min=nTime_min, split='val')

    nrois = train_data.__getitem__(0)[0].shape[0] # Trick to get the nrois (=nchannels)
    N_corr = int(nrois*(nrois+1)/2)
    
    if analysis_type == 'tc':
        net = Abide1DConvNet(nROIS=nrois)
    if analysis_type == 'corr':
        net = AbideCorrDense(corr_dim=N_corr)
        
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=.001, weight_decay=0)
    
    
    net.train()
    
    train_loss = []
    val_loss = []
    
    print(f'Training ...')
    for i_epoch in range(nepochs): 
        
        epoch_loss = 0.0
        for i, (tc, corr, dx) in enumerate(train_data_loader):

            if analysis_type == 'tc':
                vals = Variable(tc).type(torch.FloatTensor)
            if analysis_type == 'corr':
                vals = Variable(corr).type(torch.FloatTensor)
            
            dx = Variable(dx).type(torch.LongTensor)

            # forward pass
            output = net(vals)

            # calculate loss
            loss = criterion(output, torch.max(dx,1)[1])

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss

        epoch_train_loss = epoch_loss/i
        epoch_val_loss = validate_model(net, val_data, analysis_type, nTime_min=nTime_min, criterion=criterion)
        
        train_loss.append(epoch_train_loss)
        val_loss.append(epoch_val_loss)
        
        if verbose and i_epoch%1 == 0:
            print('Epoch:{} --- Train_loss:{} --- Val_loss:{}'.format(i_epoch, epoch_train_loss, epoch_val_loss))
            
    return net, train_loss, val_loss

In [41]:
def test_model(net, analysis_type, nTime_min=150):
    
    test_data = AbideData(atlas_name='schaefer_100', nTime_min=nTime_min, split='test')
    test_data_loader = DataLoader(test_data, batch_size=16, shuffle=True)
    
    net.eval()
    tot_acc = 0.0
    debug_out = []
    for i, (tc, corr, dx) in enumerate(test_data_loader):

            if analysis_type == 'tc':
                vals = Variable(tc).type(torch.FloatTensor)
            if analysis_type == 'corr':
                vals = Variable(corr).type(torch.FloatTensor)

            dx = Variable(dx).type(torch.LongTensor)
            
            # forward pass
            output = net(vals)

            debug_out.append(output)
            # calculate accuracy
            tot_acc += sum((torch.argmax(output,1) == torch.argmax(dx,1)).type(torch.FloatTensor))
            
    return tot_acc/test_data.__len__(), debug_out

In [ ]:
trained_net, training_loss, val_loss = train_network(atlas_name='schaefer_100', bptf=True, confounds=True, analysis_type='corr', nepochs=5)